In [27]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sklearn
from scipy import ndimage, misc
import math
%matplotlib inline
import cv2
import time
import glob

img_list = []
filenamelist= []
for img in glob.glob("0525_images/CCD-3/*.jpg"):
    #img_list.append(cv2.imread(img,0))
    filenamelist.append(img)
np.array(filenamelist).shape

(144,)

In [30]:
for iter_file in range(len(filenamelist)):
    
    #==============read in file==============================
    start = time.time()
    filename = filenamelist[iter_file]
    print(filename)
    image = np.array(cv2.imread(filename,0))
    backtorgb = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    
    #==============use canny to find the contour
#     image_blur = cv2.blur(image,(9,9))
#     canny = cv2.Canny(image_blur,5,20)
    ret, thresh1 = cv2.threshold(image, 70, 255, cv2.THRESH_BINARY_INV)
    open_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(4,3))
    thresh1 = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, open_kernel)

    # ===========cut the ROI by the canny edge image================
    _ , contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    approx_list = []
    index = []
    for i in range(np.array(contours).shape[0]):
        if (hierarchy[0][i][3] >= 0):
            if(cv2.contourArea(contours[i])>300000 and cv2.contourArea(contours[i])<500000):
                epsilon = 0.005# * cv2.arcLength(contours[i],True)
                index.append(i)
                approx = cv2.approxPolyDP(contours[i], epsilon, True)
                approx_list.append(approx)
    # ===========mask the image to extract the inner circle================

    oringmask = np.zeros_like(image)
    cv2.drawContours(oringmask, [approx_list[0]],-1,(255,255,255),-1)

    image[oringmask == 0] =255
    #============find the outer circle of the ROI
    ret, thresh1 = cv2.threshold(image, 110, 255, cv2.THRESH_BINARY_INV)
    c,r = cv2.minEnclosingCircle(approx_list[0])
    r = int(r)
    cx = int(c[0])
    cy = int(c[1])

    #========================line from circle center
        
        #============preprocessing
    image = cv2.blur(image,(3,3))
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    image = cv2.filter2D(image, -1, kernel)

    ret, image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY_INV)

    open_kernel = np.ones((2,3),np.uint8)
    image = cv2.morphologyEx(image, cv2.MORPH_OPEN, open_kernel)
    
        #============create 0.5 degree a line
    const = np.pi/180
    x = []
    y = []
    inner_x = []
    inner_y = []

    for degree in range(360):

        now_x = int((r+2)*np.sin(degree*const))+cx
        now_y = int((r+2)*np.cos(degree*const))+cy
        now_inner_x = int((r-30)*np.sin(degree*const))+cx
        now_inner_y = int((r-30)*np.cos(degree*const))+cy

        if(degree>0):
            prev_x = x[-1]
            prev_y = y[-1]
            prev_inner_x = inner_x[-1]
            prev_inner_y = inner_y[-1]

            x.append((now_x+prev_x)//2)
            y.append((now_y+prev_y)//2)
            inner_x.append((now_inner_x+prev_inner_x)//2)
            inner_y.append((now_inner_y+prev_inner_y)//2)

        x.append(now_x)
        y.append(now_y)
        inner_x.append(now_inner_x)
        inner_y.append(now_inner_y)
        
        #============shot from center
        
    continuous = 0 #如果有連續幾個<15才算
    for degree in range(719):
        points_on_line = np.linspace((inner_x[degree], inner_y[degree]),(x[degree],y[degree]) ,200,dtype = int)
        pass_list = image[points_on_line[:,1],points_on_line[:,0]]

        if(len(pass_list[pass_list==255])<15):
            continuous+=1#連續幾個<15
            
            if(continuous>2):
                continuous = 0
                cv2.line(backtorgb,(inner_x[degree],inner_y[degree]),(x[degree],y[degree]),255,2)
                cv2.circle(backtorgb,(x[degree],y[degree]),30, (0,255 ,255), 5)
        
        else:#沒有連續
            continuous = 0
    
    end = time.time()
    print(end - start)
    print(".\\Detect\\Stop3\\" +filenamelist[iter_file][18:-4]+'_Detect.jpg')
    cv2.imwrite(".\\Detect\\Stop3\\" +filenamelist[iter_file][18:-4]+'_Detect.jpg',backtorgb)
    

0525_images/CCD-3\1.jpg
0.14525222778320312
.\Detect\Stop3\1_Detect.jpg
0525_images/CCD-3\10.jpg
0.1477818489074707
.\Detect\Stop3\10_Detect.jpg
0525_images/CCD-3\100.jpg
0.14718127250671387
.\Detect\Stop3\100_Detect.jpg
0525_images/CCD-3\101.jpg
0.20644879341125488
.\Detect\Stop3\101_Detect.jpg
0525_images/CCD-3\102.jpg
0.1545867919921875
.\Detect\Stop3\102_Detect.jpg
0525_images/CCD-3\103.jpg
0.1740717887878418
.\Detect\Stop3\103_Detect.jpg
0525_images/CCD-3\104.jpg
0.15692353248596191
.\Detect\Stop3\104_Detect.jpg
0525_images/CCD-3\105.jpg
0.14236068725585938
.\Detect\Stop3\105_Detect.jpg
0525_images/CCD-3\106.jpg
0.16106319427490234
.\Detect\Stop3\106_Detect.jpg
0525_images/CCD-3\107.jpg
0.1535625457763672
.\Detect\Stop3\107_Detect.jpg
0525_images/CCD-3\108.jpg
0.1655564308166504
.\Detect\Stop3\108_Detect.jpg
0525_images/CCD-3\109.jpg
0.1545870304107666
.\Detect\Stop3\109_Detect.jpg
0525_images/CCD-3\11.jpg
0.15957355499267578
.\Detect\Stop3\11_Detect.jpg
0525_images/CCD-3\110.jpg


0.19447875022888184
.\Detect\Stop3\67_Detect.jpg
0525_images/CCD-3\68.jpg
0.1795196533203125
.\Detect\Stop3\68_Detect.jpg
0525_images/CCD-3\69.jpg
0.17253851890563965
.\Detect\Stop3\69_Detect.jpg
0525_images/CCD-3\7.jpg
0.15160059928894043
.\Detect\Stop3\7_Detect.jpg
0525_images/CCD-3\70.jpg
0.15609502792358398
.\Detect\Stop3\70_Detect.jpg
0525_images/CCD-3\71.jpg
0.14763426780700684
.\Detect\Stop3\71_Detect.jpg
0525_images/CCD-3\73.jpg
0.17453360557556152
.\Detect\Stop3\73_Detect.jpg
0525_images/CCD-3\74.jpg
0.15259242057800293
.\Detect\Stop3\74_Detect.jpg
0525_images/CCD-3\75.jpg
0.19843578338623047
.\Detect\Stop3\75_Detect.jpg
0525_images/CCD-3\76.jpg
0.15704059600830078
.\Detect\Stop3\76_Detect.jpg
0525_images/CCD-3\77.jpg
0.20995092391967773
.\Detect\Stop3\77_Detect.jpg
0525_images/CCD-3\78.jpg
0.18949460983276367
.\Detect\Stop3\78_Detect.jpg
0525_images/CCD-3\79.jpg
0.16954565048217773
.\Detect\Stop3\79_Detect.jpg
0525_images/CCD-3\80.jpg
0.1715693473815918
.\Detect\Stop3\80_Dete